<a href="https://colab.research.google.com/github/abigail-mcdonald/ecg_project/blob/main/12_lead_ecg_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#install and import libraries

!pip install wfdb
!pip install biosppy

import os
import wfdb as wf
import numpy as np
import matplotlib.pyplot as plt
import glob
import scipy.io
from scipy import signal
from biosppy.signals import ecg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 KB 4.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 KB 2.8 MB/s eta 0:00:00


In [2]:
!git clone https://github.com/abigail-mcdonald/ecg_project.git

Cloning into 'ecg_project'...
remote: Enumerating objects: 110, done.
remote: Total 110 (delta 0), reused 0 (delta 0), pack-reused 110
Receiving objects: 100% (110/110), 2.43 MiB | 4.45 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [3]:
!ls

drive  ecg_project  sample_data


In [4]:
path = "/content/ecg_project/data"
files = os.listdir(path)

In [5]:
#create a function to load and process hea files
def load_hea(filename):
    with open(filename, "r") as f:
        header = {}
        for line in f:
            line_values = line.strip().split(" ")
            key = line_values[0]
            values = line_values[1:]
            header[key] = values
    return header

In [6]:
#append .hea and .mat data from each file
ecg_data = []
headers = []
for file in files:
    if file.endswith(".mat"):
        mat = scipy.io.loadmat(os.path.join(path, file))
        ecg_data.append(mat)
    elif file.endswith(".hea"):
        header = load_hea(os.path.join(path, file))
        headers.append(header)

In [7]:
# Combine the ECG data and headers into a single data structure
ecg = {"data": ecg_data, "header": headers}

In [8]:
ecg

{'data': [{'val': array([[ -49,  -49,  -49, ...,  224,  190,  117],
          [-137, -137, -137, ...,  -49,  -73, -127],
          [ -88,  -88,  -88, ..., -273, -264, -244],
          ...,
          [  78,   78,   78, ...,  439,  439,  459],
          [ 224,  224,  224, ...,  400,  400,  415],
          [3387, 3387, 3387, ...,  371,  366,  386]], dtype=int16)},
  {'val': array([[ -156,  -132,   -98, ...,   -34,   -34,    10],
          [  -93,   -93,   -93, ...,   122,   132,   137],
          [   63,    39,     5, ...,   156,   166,   127],
          ...,
          [  -63,   -73,   -93, ...,    63,   102,   151],
          [ -761,  -761,  -776, ...,    15,    59,   112],
          [-3855, -3865, -3889, ...,   181,   210,   264]], dtype=int16)},
  {'val': array([[   5,   20,   34, ...,  -78,  -93,  -88],
          [-117, -112, -102, ...,  -24,  -34,  -24],
          [-122, -132, -137, ...,   54,   59,   63],
          ...,
          [ -73,  -73,  -78, ..., -112, -112, -122],
          

Back to drawing board (the way it is loaded it is not processable)
1. clone github repository
2. process .mat files separately from .hea files
3. create a variable with leads
4. use a dictonary to append .mat data to appropriate leads

In [19]:
def process_ecg(ecg_data, fs):
    if "signal" not in ecg_data:
        raise KeyError("The ECG data must contain a 'signal' key")

    if "fs" not in ecg_data:
        ecg_data["fs"] = fs

    out = bs.ecg.ecg(signal=ecg_data["signal"], sampling_rate=ecg_data["fs"])
    qrs_peaks = out["rpeaks"]
    qrs_intervals = np.diff(qrs_peaks) / ecg_data["fs"]
    return qrs_peaks, qrs_intervals

qrs_peaks, qrs_intervals = process_ecg(ecg, 500)

KeyError: ignored

Next Steps:

Signal filtering: Remove high-frequency noise from the signals using digital filters.

Baseline wander correction: Correct for baseline wander, which is a slow drift in the ECG signal baseline caused by muscle contractions and breathing.

R-peak detection: Detect the R-peak of each QRS complex, which is the highest point in the QRS complex and marks the beginning of ventricular depolarization.

ECG segmentation: Segment the ECG signals into different morphological components, such as the P-wave, QRS complex, and T-wave.

Feature extraction: Extract relevant features from the ECG signals, such as the mean, standard deviation, and heart rate variability.

Data normalization: Normalize the ECG signals so that they have a mean of zero and a standard deviation of one.

###Extra code cells as needed

In [22]:
#class of functions to find a 10 sec segment of ecg, courtesy of https://www.kaggle.com/code/nelsonsharma/ecg-02-ecg-signal-pre-processing/notebook#ECG-Pre-Processing

from scipy.io import loadmat

#global variables
g_DATA = '.mat'
g_HEAD = '.hea'
g_leads = ['I','II','III','aVR','aVL','aVF','V1','V2','V3','V4','V5','V6' ]

class ecg_segment:
    # represent a 10 sec signal of ecg from 12 leads
    
    def __init__(self, seg_name):
        self.name = seg_name
        self.file_mat = os.path.join(path,self.name+'.mat')
        self.file_hea = os.path.join(path,self.name+'.hea')
        self.load_data()
        self.load_header()
        
        
    def load_data(self):
        self.signals=loadmat(self.file_mat)['val'].astype(np.float64)
        self.signald = {}
        for i in range(0,len(g_leads)):
            self.signald[g_leads[i]] = self.signals[i,:]
        # an array of shape (12, 5000) each row is a lead in g_leads
        return
    
    def load_header(self):
        with open(self.file_hea,'r') as f:
            hlines=f.readlines() # each line ends with newline char
            #JS34506 12 500 5000 28-Mar-2021 04:58:01  
            #0    (6x1) < from row0 extract everything
            tl = (hlines[0][0:-1]).split(" ")
            assert (tl[0] == self.name)
            assert (tl[1] == '12')
            assert (tl[2] == '500')
            assert (tl[3] == '5000')
            #self.timestamp = datetime.datetime.strptime("%d-%b-%Y_%H:%M:%S", tl[4]+'_'+tl[5])
            for i in range(1,12):
                assert ((hlines[i][0:-1]).split(" ")[8] == g_leads[i-1])
                assert ((hlines[i][0:-1]).split(" ")[8] == g_leads[i-1])
                
            self.age = int(hlines[13][0:-1].split(" ")[1])
            self.gender = hlines[14][0:-1].split(" ")[1]
            self.Dx = (hlines[15][0:-1].split(" ")[1])
            self.Rx = (hlines[16][0:-1].split(" ")[1])
            self.Hx = (hlines[17][0:-1].split(" ")[1])
            self.Sx = (hlines[18][0:-1].split(" ")[1])
            
        return 
    
    def info(self):
        print(self.name,self.gender,self.age)
        return

In [29]:
rec_list = list(ecg['data'])
recn = np.random.choice(rec_list, size=1)[0]  # total records = rec_count
ecg_seg = ecg_segment(recn)
ecg_seg.info()

TypeError: ignored

,data,header
0,"{'val': [[-5, -29, -54, -68, -49, -34, -54, -5...","{'JS00012': ['12', '500', '5000', '23-Mar-2021..."
1,"{'val': [[-39, -49, -49, -49, -54, -44, -39, -...","{'JS00013': ['12', '500', '5000', '23-Mar-2021..."
2,"{'val': [[-44, -49, -44, -49, -44, -49, -54, -...","{'JS00048': ['12', '500', '5000', '23-Mar-2021..."
3,"{'val': [[-63, -44, -39, 0, 39, 63, 132, 205, ...","{'JS00008': ['12', '500', '5000', '23-Mar-2021..."
4,"{'val': [[68, 73, 63, 54, 39, 24, 34, 24, 15, ...","{'JS00041': ['12', '500', '5000', '23-Mar-2021..."


In [19]:
import biosppy.signals as bs

out = biosppy.signals.ecg.christov_segmenter(ecg["data"], sampling_rate=ecg["fs"])
qrs_peaks = out[0]
qrs_intervals = np.diff(qrs_peaks) / ecg["fs"]

AttributeError: ignored

In [4]:
# load dataset

DATASET_DIR = '/content/ecg_project/data'
print('DATASET DIR ::',DATASET_DIR)

DATASET DIR :: /content/ecg_project/data


In [13]:
files = os.listdir(DATASET_DIR)

headers = []
for file in files:
    if file.endswith(".hea"):
      header = load_hea(os.path.join(DATASET_DIR, file))
      headers.append(header)

In [6]:
record = wf.rdrecord('JS00001', channels=[0])
wf.plotrec(record, title='Record JS00001')

FileNotFoundError: ignored

In [2]:
def load_hea(filename):
    with open(filename, "r") as f:
        # Parse the header information from the .hea file
        header = {}
        for line in f:
            key, value = line.strip().split(" ")
            header[key] = value
    return header

In [ ]:
# create variables

In [ ]:
# create records

In [ ]:
# plot sample records

In [ ]:
# process: denoise, detect R peaks, remove baseline wander

In [ ]:
# turn into csv file?

In [5]:
file_pattern = '*.mat'
file_list = glob.glob(file_pattern)

for file_name in file_list:
    record = wf.rdrecord(file_name)
    signal = record.p_signal
    # Perform processing on the loaded signal

In [15]:
def process_ecg(ecg_data, fs):
    if "fs" not in ecg_data:
        ecg_data["fs"] = fs

    ann = wf.rdann("sample", "atr", sampfrom=0, sampto=None)
    qrs_peaks = ann.sample
    qrs_intervals = np.diff(qrs_peaks) / ecg_data["fs"]
    return qrs_peaks, qrs_intervals

qrs_peaks, qrs_intervals = process_ecg(ecg, 500)

FileNotFoundError: ignored